In [4]:
%load_ext autoreload
%autoreload 2

import os, json
import nibabel as nb

from keras import metrics 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LocallyConnected1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from keras.utils.vis_utils import plot_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import hrf_estimation
import scipy.signal as signal

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
!pwd


/


In [135]:
os.chdir('/Users/knapen/projects/prfpy/prfpy/')
from gauss import *
from timecourse import *
from stimulus import PRFStimulus